In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 13:03:06 WARN Utils: Your hostname, MarlonPenuela resolves to a loopback address: 127.0.1.1; using 172.18.73.232 instead (on interface eth0)
24/03/03 13:03:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 13:03:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_green.rdd.take(10)

[Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 8, 9, 42), lpep_dropoff_datetime=datetime.datetime(2020, 1, 21, 8, 24, 52), store_and_fwd_flag='N', RatecodeID=1, PULocationID=7, DOLocationID=82, passenger_count=1, trip_distance=2.9, fare_amount=14.0, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=14.8, payment_type=2, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 19, 42, 20), lpep_dropoff_datetime=datetime.datetime(2020, 1, 28, 19, 51, 10), store_and_fwd_flag='N', RatecodeID=1, PULocationID=95, DOLocationID=196, passenger_count=1, trip_distance=1.35, fare_amount=7.5, extra=1.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=9.3, payment_type=2, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 18, 37, 21), lpep_dropoff_datet

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [13]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [5]:
rows = rdd.take(10)
row = rows[0]

In [6]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 8, 9, 42), PULocationID=7, total_amount=14.8)

In [7]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [8]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [9]:
from collections import namedtuple

In [10]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [11]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [15]:
from pyspark.sql import types

In [16]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [18]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [20]:
df_result.show(5)

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-21 08:00:00|   7|327.32000000000016|   28|
|2020-01-08 12:00:00|  42|162.56000000000003|   19|
|2020-01-18 01:00:00| 166|118.49999999999999|    6|
|2020-01-16 05:00:00| 117|            103.79|    2|
|2020-01-14 00:00:00|  82|294.83000000000004|   19|
+-------------------+----+------------------+-----+
only showing top 5 rows



In [23]:
df_result.write.parquet('tmp/green-revenue', mode='overwrite' )

24/03/03 13:37:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [33]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [34]:
import pandas as pd

In [35]:
rows = duration_rdd.take(10)

In [36]:
df = pd.DataFrame(rows, columns=columns)

In [37]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [38]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [39]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [40]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

/home/mlops/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/mlops/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [41]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|              14.5|
|              6.75|
|              31.0|
|               8.2|
|5.3500000000000005|
|              9.45|
|               3.1|
|              3.75|
|              10.0|
|              5.45|
|3.4499999999999997|
|              79.4|
|              14.0|
|               5.0|
|              3.85|
|             35.65|
|45.300000000000004|
| 5.300000000000001|
|             13.55|
| 85.85000000000001|
+------------------+
only showing top 20 rows

